In [2]:
from nansat import Nansat
from osgeo import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import scipy
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

import matplotlib as mpl
# mpl.use('Qt5Agg')
import matplotlib.pyplot as plt

In [115]:
s2_path = r'C:/Users/ariane/shared/NERSC/classification/Subset_S2_20210716_resampled.tif'
rpoints = 'C:/Users/ariane/shared/NERSC/classification/rasterpoints.shp'
path = 'C:/Users/ariane/shared/NERSC/classification/'

In [116]:
points = gpd.read_file(rpoints)
class_names = points['classtxt'].unique()
print(class_names)
points

['Water' 'Seabed' 'Road' 'Pioneer' 'LSM' 'USM' 'Agriculture']


,classtxt,classid,B2,B3,B4,B8,B11,B12,NDVI,SAVI,MSAVI,NDMI,EVI,AWEI,NDWI,geometry
0,Water,1,244.0,376.0,252.0,148.0,172.0,147.0,-0.260000,2.758518,0.023151,-0.075000,-0.026450,0.037475,0.435115,POINT (8.78073 54.56462)
1,Water,1,231.0,410.0,279.0,126.0,152.0,124.0,-0.377778,2.746901,0.018096,-0.093525,-0.037994,0.065950,0.529851,POINT (8.79207 54.56766)
2,Water,1,262.0,402.0,236.0,119.0,154.0,127.0,-0.329577,2.779272,0.017793,-0.128205,-0.030564,0.061300,0.543186,POINT (8.77722 54.56884)
3,Water,1,205.0,339.0,210.0,105.0,140.0,128.0,-0.333333,2.802446,0.015666,-0.142857,-0.026711,0.041775,0.527027,POINT (8.79249 54.57158)
4,Water,1,308.0,489.0,330.0,162.0,166.0,159.0,-0.341463,2.700655,0.023948,-0.012195,-0.042718,0.081425,0.502304,POINT (8.78216 54.57501)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Agriculture,7,130.0,522.0,138.0,5644.0,2242.0,992.0,0.952266,1.901873,0.977237,0.431397,0.888236,-1.101900,-0.830684,POINT (8.85396 54.65587)
272,Agriculture,7,355.0,886.0,728.0,3010.0,1737.0,898.0,0.610487,1.977798,0.560582,0.268169,0.387686,-0.662600,-0.545175,POINT (8.86956 54.65963)
273,Agriculture,7,338.0,924.0,827.0,2730.0,1452.0,764.0,0.535001,1.975342,0.504292,0.305595,0.313881,-0.489550,-0.494253,POINT (8.86083 54.65956)
274,Agriculture,7,388.0,952.0,894.0,2830.0,1567.0,761.0,0.519871,1.941311,0.519487,0.287241,0.316671,-0.526025,-0.496563,POINT (8.86424 54.66112)


In [117]:
with rasterio.open(s2_path) as s2: 
    meta = s2.meta
    arr = s2.read()
meta

array([[[ 157,  183,  193, ...,   77,    1,    3],
        [ 201,  183,  182, ...,    1,    1,    1],
        [ 201,  190,  209, ...,    1,   19,    6],
        ...,
        [ 199,  170,  226, ...,  138,  167,  267],
        [ 187,  214,  227, ...,  152,  144,  157],
        [ 188,  223,  244, ...,  148,  141,  164]],

       [[ 335,  320,  355, ...,  243,  137,  116],
        [ 367,  334,  335, ...,  147,  112,  141],
        [ 414,  357,  380, ...,  152,  135,  148],
        ...,
        [ 382,  386,  403, ...,  512,  620,  713],
        [ 400,  396,  421, ...,  577,  598,  588],
        [ 390,  404,  438, ...,  565,  532,  526]],

       [[ 284,  273,  315, ...,  333,  101,  101],
        [ 299,  284,  297, ...,  140,   88,   72],
        [ 315,  311,  317, ...,  139,  114,  108],
        ...,
        [ 181,  182,  191, ...,  190,  253,  339],
        [ 183,  189,  213, ...,  173,  200,  240],
        [ 183,  205,  238, ...,  243,  258,  274]],

       [[1009, 1019,  930, ..., 2156,

In [157]:
### Preparation for raster stack of all s2 scenes and indices - extract rasterpoint values for all layers of the stack
### maybe write in loop for this amount of layers... ?

### Reproject shapefile to S2 
points = points.to_crs(meta["crs"])
coord_list = [(x,y) for x,y in zip(points['geometry'].x , points['geometry'].y)]

### open the raster 
with rasterio.open(s2_path) as s2: 
    samplevalues = [x for x in s2.sample(coord_list)]
    
B2 = [i[0] for i in samplevalues] 
B3 = [i[1] for i in samplevalues]
B4 = [i[2] for i in samplevalues]
B8 = [i[3] for i in samplevalues]
B11 = [i[4] for i in samplevalues]
B12 = [i[5] for i in samplevalues]

points['B2']=B2
points['B3']=B3
points['B4']=B4
points['B8']=B8
points['B11']=B11
points['B12']=B12

points.columns



Index(['classtxt', 'classid', 'B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'NDVI',
       'SAVI', 'MSAVI', 'NDMI', 'EVI', 'AWEI', 'NDWI', 'geometry', 'B5'],
      dtype='object')

In [186]:
# points.to_file('C:/Users/ariane/shared/NERSC/classification/rasterpoints.csv', driver="CSV")
cols = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'] # 'NDVI', 'SAVI', 'MSAVI', 'NDMI', 'EVI', 'AWEI', 'NDWI']
scaler = StandardScaler()

### Splitting the training points into training and testing data for the RandomForest Model
X_train, X_test, y_train, y_test=train_test_split(points[cols], points['classid'], train_size=0.7, random_state=42)
X_train
print(y_train)

### Trying to scale data 

X_scaled = scaler.fit_transform(X_train[cols].to_numpy())
X_scaled

# Y_scaled = scaler.fit_transform(y_train[cols].to_numpy())
# Y_scaled

# X_scaled = scaler.fit_transform(X_train)
# Y_scaled = scaler.fit_transform(y_train)

### Trying PCA transformation of training data to enhance results

# X_train.corr()
# pc = PCA().fit(X_train)
# pc_train = pc.transform(X_train)
# pc_test = pc.transform(X_test)

38     2
259    7
112    3
57     2
97     3
      ..
188    6
71     2
106    3
270    7
102    3
Name: classid, Length: 193, dtype: int64


In [120]:
# X, y = make_classification(random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# pipe = make_pipeline(StandardScaler(), LogisticRegression())
# pipe.fit(X_train, y_train)  # apply scaling on training data
# Pipeline(steps=[('standardscaler', StandardScaler()),
#                ('logisticregression', LogisticRegression())])

# pipe.score(X_test, y_test)  # apply scaling on testing data, without leaking training data.


In [193]:
### Trying to write scaling into classification

# clf = make_pipeline(StandardScaler(),
#                     RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=50))
                    

### Normal classification without scaling
clf = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=50)

### Alternative classification: Support Vector
# LinearSVC(random_state=0, tol=1e-3)
                    
# clf.fit(pc_train, y_train)  # PCA model fit
                    
#clf.fit(X_train, y_train) # normal model fit

clf.fit(X_train, y_train) # scaled model fit

RandomForestClassifier(n_jobs=-1, random_state=50)

In [188]:
clf_classified = clf.predict(X_test)
#clf_classified_pca = clf.predict(pc_test)
# clf_classified.shape

C:\Users\ariane\anaconda3\envs\nansenproject2\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [123]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
# scores = cross_val_score(clf, pc_train, y_train, cv=5)
scores

array([0.84615385, 0.82051282, 0.92307692, 0.92105263, 0.94736842])

In [124]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
# user warning: X does not have valid feature names, but RFC was fitted with feature names --> ? 

# classification = clf_classified.reshape((rows, cols))

print(clf_classified)

# output = 'C:/Users/ariane/shared/NERSC/classification/outputRFC_w.tif'

0.89 accuracy with a standard deviation of 0.05
[1 5 6 5 6 2 5 6 5 5 2 5 2 7 5 2 7 5 3 7 2 2 5 5 7 6 3 2 1 2 1 3 1 5 5 2 6
 2 1 2 2 4 2 1 3 6 1 6 5 6 3 5 5 1 5 1 7 5 1 5 5 3 2 4 1 6 7 2 5 3 3 6 7 1
 1 3 6 3 2 3 4 6 7]


In [189]:
# np.save('/home/ariane/nansen/classification/better_classification.npy', classification)
# classification = np.load('/home/ariane/nansen/classification/better_classification.npy')


#build confusion matrix (x-axis: predicted pixels, y-axis: reference pixels; class 1,2,3,4,5)
cm = confusion_matrix(y_test, clf_classified, labels = clf.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = clf.classes_)
disp.plot()
# %matplotlib inline
plt.show()
# driver = gdal.GetDriverByName('GTiff')
# print(driver)
# rows, cols = classification.shape
# print(rows, cols)
# rasterDS = driver.Create(output, cols, rows)
# print(rasterDS)

# rasterDS.SetGeoTransform(geo_transform)
# rasterDS.SetProjection(projection)
# rasterDS.GetRasterBand(1).WriteArray(classification)
# rasterDS.GetRasterBand(1).SetNoDataValue(0)
# rasterDS.FlushCache()

In [126]:
# plot_confusion_matrix(clf, pc_test, y_test)

# cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm,
#                              display_labels=clf.classes_)

from sklearn.metrics import classification_report

In [190]:
#calculate number of pixels that were classified correctly
accuracy_score(y_test, clf_classified, normalize=False)

pd.DataFrame(classification_report(y_test, clf_classified, digits=2, output_dict=True))

C:\Users\ariane\anaconda3\envs\nansenproject2\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ariane\anaconda3\envs\nansenproject2\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ariane\anaconda3\envs\nansenproject2\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

,1,2,3,4,5,6,7,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,0.0,0.108434,0.108434,0.015491,0.011758
recall,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.108434,0.142857,0.108434
f1-score,0.0,0.0,0.0,0.0,0.0,0.0,0.195652,0.108434,0.027950,0.021215
support,12.0,17.0,11.0,9.0,12.0,13.0,9.000000,0.108434,83.000000,83.000000


In [128]:
# accuracy_score(y_test, clf_classified, normalize=True)

In [129]:
# calculate precision score per class
# precision_score(y_test, clf_classified, average=None)

In [130]:
# calculate recall score per class
# recall_score(y_test, clf_classified, average=None)

In [131]:
# arr
#take the (multi layer) raster (arr) and flatten into 1-dimensional array
# arrays_flat = [arr[i, :, :].flatten() for i in range(arr.shape[0])] 

In [194]:
### Save the model

joblib.dump(clf, path + 'random_forest.joblib')
loaded_clf = joblib.load(path+'random_forest.joblib')

# loaded_clf.predict(pc_test)
loaded_clf.predict(X_test)

array([1, 5, 6, 5, 6, 2, 5, 6, 5, 5, 2, 5, 2, 7, 5, 2, 7, 5, 3, 7, 2, 2,
       5, 5, 7, 6, 3, 2, 1, 2, 1, 3, 1, 5, 5, 2, 6, 2, 1, 2, 2, 4, 2, 1,
       3, 6, 1, 6, 5, 6, 3, 5, 5, 1, 5, 1, 7, 5, 1, 5, 5, 3, 2, 4, 1, 6,
       7, 2, 5, 3, 3, 6, 7, 1, 1, 3, 6, 3, 2, 3, 4, 6, 7], dtype=int64)

In [133]:
clf_classified

array([1, 5, 6, 5, 6, 2, 5, 6, 5, 5, 2, 5, 2, 7, 5, 2, 7, 5, 3, 7, 2, 2,
       5, 5, 7, 6, 3, 2, 1, 2, 1, 3, 1, 5, 5, 2, 6, 2, 1, 2, 2, 4, 2, 1,
       3, 6, 1, 6, 5, 6, 3, 5, 5, 1, 5, 1, 7, 5, 1, 5, 5, 3, 2, 4, 1, 6,
       7, 2, 5, 3, 3, 6, 7, 1, 1, 3, 6, 3, 2, 3, 4, 6, 7], dtype=int64)

In [134]:
meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 960,
 'height': 1091,
 'count': 6,
 'crs': CRS.from_epsg(32632),
 'transform': Affine(10.0, 0.0, 484550.0,
        0.0, -10.0, 6057160.0)}

In [135]:
#take the (multi layer) raster (arr) and flatten into 1-dimensional array
arrays_flat = [arr[i, :, :].flatten() for i in range(arr.shape[0])] 

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 960,
 'height': 1091,
 'count': 6,
 'crs': CRS.from_epsg(32632),
 'transform': Affine(10.0, 0.0, 484550.0,
        0.0, -10.0, 6057160.0)}

In [136]:
#define which band refers to which column in the raster data frame
#the order (0-X) is the same as in the original raster as it can be seen in QGIS UI
raster_df = pd.DataFrame({
    "B2" : arrays_flat[0],
    "B3" : arrays_flat[1],
    "B4" : arrays_flat[2],
    "B8" : arrays_flat[3],
    "B11" : arrays_flat[4],
    "B12" : arrays_flat[5]})

In [137]:
cols_raster_df = raster_df.columns
#show length of raster data frame to check number
len(raster_df.columns)
#show if it worked and all the column-names are correct and in the correct order in the raster data frame
raster_df

,B2,B3,B4,B8,B11,B12
0,157,335,284,1009,445,271
1,183,320,273,1019,373,229
2,193,355,315,930,373,229
3,229,421,375,873,340,208
4,195,379,350,846,340,208
...,...,...,...,...,...,...
1047355,138,456,200,5112,1436,635
1047356,165,507,230,4924,1436,635
1047357,148,565,243,4804,1250,584
1047358,141,532,258,4308,1250,584


In [138]:
raster_df.columns

Index(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], dtype='object')

In [139]:
#classify the raster
raster_pred = clf.predict(raster_df)

In [140]:
#reshape from array to raster
#take shape from original raster
shape_out = (arr.shape[1], arr.shape[2])
print(shape_out)

(1091, 960)


In [141]:
#show shape from original raster: 2-dimensional (correct? multidimensional?)
shape_out

#final classified raster is the classified raster but reshaped in the shape as the original --> multidimensional
raster_out = raster_pred.reshape(shape_out)
print(raster_out.shape)

### Output path
output = path + 'RF_July_Classification.tif'
output
#define output data type (same as original)
meta["dtype"] = 'float32'
meta["count"] = 1

#test if it worked
meta
with rasterio.open(output, 'w', **meta) as dst:
    dst.write(raster_out, 1)
    
raster_out

(1091, 960)


array([[5, 5, 5, ..., 5, 7, 5],
       [5, 5, 5, ..., 7, 7, 5],
       [5, 5, 5, ..., 5, 5, 5],
       ...,
       [1, 1, 1, ..., 7, 7, 6],
       [1, 1, 1, ..., 7, 7, 7],
       [1, 1, 1, ..., 5, 5, 5]], dtype=int64)

In [197]:
### Apply to more sentinel 2 scenes

path = 'C:/Users/ariane/shared/NERSC/classification/'

### September scene
with rasterio.open(path+'Subset_S2A_20210909_resampled.tif') as s2: 
    meta = s2.meta
    arr = s2.read()

keys = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
arrays_flat = [arr[i, :, :].flatten() for i in range(arr.shape[0])]
meta
raster_df = pd.DataFrame({
    "B2" : arrays_flat[0],
    "B3" : arrays_flat[1],
    "B4" : arrays_flat[2],
    "B8" : arrays_flat[3],
    "B11" : arrays_flat[4],
    "B12" : arrays_flat[5]})
cols_raster_df = raster_df.columns

#show length of raster data frame to check number
len(raster_df.columns)

#show if it worked and all the column-names are correct and in the correct order in the raster data frame
raster_df
raster_df.columns

#classify the raster
raster_pred = clf.predict(raster_df)
#reshape from array to raster
#take shape from original raster
shape_out = (arr.shape[1], arr.shape[2])
print(shape_out)
#show shape from original raster: 2-dimensional (correct? multidimensional?)
shape_out

#final calssified raster is the classified raster but reshaped in the shape as the original --> multidimensional
raster_out = raster_pred.reshape(shape_out)
print(raster_out.shape)

### Output path
output = path + 'RF_Sep_Classification.tif'
output
#define output data type (same as original)
meta["dtype"] = 'float32'
meta["count"] = 1

#test if it worked
meta
with rasterio.open(output, 'w', **meta) as dst:
    dst.write(raster_out, 1)
    
raster_out

(1091, 960)
(1091, 960)


array([[4, 4, 4, ..., 5, 5, 5],
       [4, 4, 4, ..., 5, 5, 5],
       [4, 4, 4, ..., 5, 5, 5],
       ...,
       [1, 1, 1, ..., 6, 7, 7],
       [1, 1, 1, ..., 6, 7, 6],
       [1, 1, 1, ..., 5, 5, 5]], dtype=int64)

In [199]:
### April scene

with rasterio.open(path+'Subset_S2_20210419_resampled.tif') as s2: 
    meta = s2.meta
    arr = s2.read()

keys = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
arrays_flat = [arr[i, :, :].flatten() for i in range(arr.shape[0])]
raster_df = pd.DataFrame({
    "B2" : arrays_flat[0],
    "B3" : arrays_flat[1],
    "B4" : arrays_flat[2],
    "B8" : arrays_flat[3],
    "B11" : arrays_flat[4],
    "B12" : arrays_flat[5]})
cols_raster_df = raster_df.columns

#show length of raster data frame to check number
len(raster_df.columns)

#show if it worked and all the column-names are correct and in the correct order in the raster data frame
raster_df
raster_df.columns

#classify the raster
raster_pred = clf.predict(raster_df)
#reshape from array to raster
#take shape from original raster
shape_out = (arr.shape[1], arr.shape[2])
print(shape_out)
#show shape from original raster: 2-dimensional (correct? multidimensional?)
shape_out

#final calssified raster is the classified raster but reshaped in the shape as the original --> multidimensional
raster_out = raster_pred.reshape(shape_out)
print(raster_out.shape)

### Output path
output = path + 'RF_April_Classification.tif'
output
#define output data type (same as original)
meta["dtype"] = 'float32'
meta["count"] = 1

#test if it worked
meta
with rasterio.open(output, 'w', **meta) as dst:
    dst.write(raster_out, 1)
    
raster_out

(1485, 1164)
(1485, 1164)


array([[2, 3, 3, ..., 3, 3, 3],
       [2, 3, 3, ..., 3, 3, 3],
       [2, 3, 3, ..., 3, 3, 3],
       ...,
       [1, 1, 1, ..., 7, 7, 7],
       [1, 1, 1, ..., 7, 7, 7],
       [1, 1, 1, ..., 7, 7, 7]], dtype=int64)

In [201]:
### May

with rasterio.open(path+'/subsets/Subset_S2A_20210509_resampled.tif') as s2: 
    meta = s2.meta
    arr = s2.read()

keys = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
arrays_flat = [arr[i, :, :].flatten() for i in range(arr.shape[0])]
raster_df = pd.DataFrame({
    "B2" : arrays_flat[0],
    "B3" : arrays_flat[1],
    "B4" : arrays_flat[2],
    "B8" : arrays_flat[3],
    "B11" : arrays_flat[4],
    "B12" : arrays_flat[5]})
cols_raster_df = raster_df.columns

#show length of raster data frame to check number
len(raster_df.columns)

#show if it worked and all the column-names are correct and in the correct order in the raster data frame
raster_df
raster_df.columns

#classify the raster
raster_pred = clf.predict(raster_df)
#reshape from array to raster
#take shape from original raster
shape_out = (arr.shape[1], arr.shape[2])
print(shape_out)
#show shape from original raster: 2-dimensional (correct? multidimensional?)
shape_out

#final calssified raster is the classified raster but reshaped in the shape as the original --> multidimensional
raster_out = raster_pred.reshape(shape_out)
print(raster_out.shape)

### Output path
output = path + 'RF_May_Classification.tif'
output
#define output data type (same as original)
meta["dtype"] = 'float32'
meta["count"] = 1

#test if it worked
meta
with rasterio.open(output, 'w', **meta) as dst:
    dst.write(raster_out, 1)
    
raster_out

(1091, 960)
(1091, 960)


array([[2, 2, 2, ..., 5, 5, 5],
       [2, 2, 2, ..., 5, 5, 5],
       [2, 2, 2, ..., 5, 5, 5],
       ...,
       [2, 2, 2, ..., 3, 3, 3],
       [2, 2, 2, ..., 3, 3, 3],
       [2, 2, 2, ..., 7, 7, 7]], dtype=int64)

In [202]:
### October

with rasterio.open(path+'/subsets/Subset_S2B_20211024_resampled.tif') as s2: 
    meta = s2.meta
    arr = s2.read()

keys = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
arrays_flat = [arr[i, :, :].flatten() for i in range(arr.shape[0])]
raster_df = pd.DataFrame({
    "B2" : arrays_flat[0],
    "B3" : arrays_flat[1],
    "B4" : arrays_flat[2],
    "B8" : arrays_flat[3],
    "B11" : arrays_flat[4],
    "B12" : arrays_flat[5]})
cols_raster_df = raster_df.columns

#show length of raster data frame to check number
len(raster_df.columns)

#show if it worked and all the column-names are correct and in the correct order in the raster data frame
raster_df
raster_df.columns

#classify the raster
raster_pred = clf.predict(raster_df)
#reshape from array to raster
#take shape from original raster
shape_out = (arr.shape[1], arr.shape[2])
print(shape_out)
#show shape from original raster: 2-dimensional (correct? multidimensional?)
shape_out

#final calssified raster is the classified raster but reshaped in the shape as the original --> multidimensional
raster_out = raster_pred.reshape(shape_out)
print(raster_out.shape)

### Output path
output = path + 'RF_Oct_Classification.tif'
output
#define output data type (same as original)
meta["dtype"] = 'float32'
meta["count"] = 1

#test if it worked
meta
with rasterio.open(output, 'w', **meta) as dst:
    dst.write(raster_out, 1)
    
raster_out

(1091, 960)
(1091, 960)


array([[2, 2, 2, ..., 5, 5, 5],
       [2, 2, 2, ..., 5, 5, 5],
       [2, 2, 2, ..., 5, 5, 5],
       ...,
       [2, 2, 2, ..., 3, 3, 3],
       [1, 2, 2, ..., 3, 3, 3],
       [1, 2, 2, ..., 3, 3, 3]], dtype=int64)